In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from config import props, filenames
import analysis

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [3]:
SAVE_FIGS = False

In [4]:
data_raw = pd.read_pickle(filenames.data+'data.pkl')
data_raw.t = data_raw.t - data_raw.t[0]
data_convoluted = analysis.data_cleaning(data_raw.copy())
pert_times = analysis.find_perts(data_convoluted.copy())
data = analysis.data_interpolation(data_convoluted.copy(), pert_times)

In [5]:
plt.figure()
plt.plot(data_raw.t, data_raw.I, label='raw')
plt.plot(data.t, data.I, label='treated')
plt.legend(loc=1)
plt.show()
if SAVE_FIGS:
    plt.savefig(filenames.notes+'current_waveform.png')

In [6]:
cycles = analysis.find_cycles(data, pert_times)

Warning! Some info might be wrong/missing
Empty DataFrame
Columns: [start, duration, expected_duration, had_pert]
Index: []
            start     duration  expected_duration  had_pert
593  34448.255152  8592.433616          85.456184     False


In [7]:
perts = analysis.pert_response(data, cycles, pert_times[:-1])

In [8]:
fig, axs = plt.subplots(2, sharex=True)

axs[0].plot(cycles['start'], cycles['duration'])
axs[0].plot(perts.time, perts.expected_period)
axs[0].set_ylabel('period')
axs[0].scatter(cycles['start'][cycles['had_pert']], cycles['duration'][cycles['had_pert']], c='r', s=10)

axs[1].scatter(perts['time'], perts['phase'], marker = 'x')
# axs[1].plot(perts['time'], perts['phase'])
axs[1].set_ylabel('pert. phase')

fig.supxlabel('time [s]')
fig.tight_layout()
if SAVE_FIGS:
    fig.savefig(filenames.notes+'period_vs_time.png')

In [9]:
def on_pick(event) -> tuple[mpl.figure.Figure, mpl.figure.Axes]:
    pert_time = perts.time.iloc[event.ind[0]]
    print(event.ind[0])
    fig, ax = plt.subplots()
    data_before = data[(data.t > pert_time - props.max_period) & (data.t < pert_time)]
    data_after = data[(data.t > pert_time) & (data.t < pert_time + props.max_period)]

    ax.plot(data_before.I, data_before.emsi)
    ax.plot(data_after.I, data_after.emsi)
    ax.scatter(data_before.I.iloc[-1], data_before.emsi.iloc[-1], c='r')
    fig.suptitle(rf'Perturbation at $\phi$ = {perts.phase.iloc[event.ind[0]]:.2f} -- phase space')
    fig.supxlabel(r'current [mA/cm$^2$]')
    fig.supylabel(r'emsi signal [$\xi$]')
    fig.show()

    fig, ax = plt.subplots()
    ax.plot(data_before.t, data_before.I)
    ax.plot(data_after.t, data_after.I)
    ax.scatter(data_before.t.iloc[-1], data_before.I.iloc[-1], c='r')
    fig.suptitle(rf'Perturbation at $\phi$ = {perts.phase.iloc[event.ind[0]]:.2f} -- current')
    fig.supxlabel('time [s]')
    fig.supylabel(r'current [mA/cm$^2$]')
    fig.show()

In [10]:
if props.pert_type == 'U':
    title = f'PRC for {props.voltage}{'+' if props.pert_strength>0 else '-'}{props.pert_strength}V, {props.pert_dt}s'
else:
    title = f'PRC for {props.voltage}, {'+' if props.pert_strength>0 else '-'}{props.pert_dt}s light'
new_fig, new_ax = plt.subplots()
new_fig.suptitle(title)
new_ax.axhline(0)
new_ax.scatter(perts['phase'], perts['response'], c=perts['time'], picker=True)
id = new_fig.canvas.mpl_connect('pick_event', on_pick)
# if SAVE_new_figS:
#     new_fig.save_fig(filenames.notes+'prc.png')

In [ ]:
new_fig.canvas.mpl_disconnect(14)

In [ ]:
fig, axs = plt.subplots(2,2, sharex = True, sharey=True, figsize = (14, 9))
things_to_plot = ['basis', 'response_0', 'response_1', 'response_2']
for (thing, ax) in zip(things_to_plot, axs.flatten()):
    ax.scatter(perts['phase'], perts[thing], c=perts['time'])
    ax.set_title(thing)
    ax.axhline(0, ls='--')
fig.suptitle(title)
fig.supxlabel(r'$\phi$')
fig.supylabel(r'$\Delta\phi$')
fig.tight_layout()
if SAVE_FIGS:
    fig.savefig(filenames.notes+'prc_full.png')